# day 15

https://adventofcode.com/15/day/15

In [ ]:
import logging
import logging.config
import os

import yaml

In [ ]:
with open('../logging.yaml') as fp:
    logging_config = yaml.load(fp, Loader=yaml.FullLoader)

logging.config.dictConfig(logging_config)

In [ ]:
FNAME = os.path.join('data', 'day15.txt')

LOGGER = logging.getLogger('day15')

## part 1

### problem statement:

#### loading data

In [ ]:
test_data = "rn=1,cm-,qp=3,cm=2,qp-,pc=4,ot=9,ab=5,pc-,pc=6,ot=7"

In [ ]:
def load_data(fname=FNAME):
    with open(fname) as fp:
        return fp.read().strip()

#### function def

In [ ]:
def hash(s: str, start_val: int = 0) -> int:
    current_val = start_val
    for char in s:
        current_val += ord(char)
        current_val *= 17
        current_val %= 256
    return current_val

assert hash('HASH') == 52
assert hash("rn=1") == 30
assert hash("cm-") == 253
assert hash("qp=3") == 97
assert hash("cm=2") == 47
assert hash("qp-") == 14
assert hash("pc=4") == 180
assert hash("ot=9") == 9
assert hash("ab=5") == 197
assert hash("pc-") == 48
assert hash("pc=6") == 214
assert hash("ot=7") == 231

In [ ]:
def parse_data(s: str) -> list[str]:
    return s.replace('\n', '').split(',')

In [ ]:
def q_1(data):
    return sum(hash(s) for s in parse_data(data))

#### tests

In [ ]:
def test_q_1():
    LOGGER.setLevel(logging.DEBUG)
    assert q_1(test_data) == 1320
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_1()

#### answer

In [ ]:
q_1(load_data())

## part 2

### problem statement:

#### function def

In [ ]:
from dataclasses import dataclass
from enum import Enum

class Operation(str, Enum):
    EQUALS = '='
    DASH = '-'


@dataclass
class Instruction:
    lens_label: str
    operation: Operation
    focal_length: int = None


def parse_instruction(instruction: str) -> Instruction:
    try:
        lens_label, focal_length = instruction.split('=')
        focal_length = int(focal_length)
        return Instruction(lens_label=lens_label, operation=Operation.EQUALS, focal_length=focal_length)
    except ValueError:
        return Instruction(lens_label=instruction[:-1], operation=Operation.DASH)
    raise ValueError()

assert parse_instruction("rn=1") == Instruction(lens_label='rn', operation=Operation.EQUALS, focal_length=1)
assert parse_instruction("rn-") == Instruction(lens_label='rn', operation=Operation.DASH)

In [ ]:
def focusing_power(box_idx: int, slot_idx: int, focal_length: int) -> int:
    return (box_idx + 1) * (slot_idx + 1) * focal_length

assert focusing_power(box_idx=0, slot_idx=0, focal_length=1) == 1
assert focusing_power(box_idx=0, slot_idx=1, focal_length=2) == 4
assert focusing_power(box_idx=3, slot_idx=0, focal_length=7) == 28
assert focusing_power(box_idx=3, slot_idx=1, focal_length=5) == 40
assert focusing_power(box_idx=3, slot_idx=2, focal_length=6) == 72

In [ ]:
class Boxes(dict):
    def add_to_box(self, box_idx: int, lens_label: str, focal_length: int) -> None:
        if box_idx not in self:
            self[box_idx] = {}
        self[box_idx][lens_label] = focal_length

    def remove_from_box(self, box_idx: int, lens_label: str) -> None:
        try:
            _ = self[box_idx].pop(lens_label)
        except:
            pass

    def process_instruction(self, instruction: Instruction) -> None:
        box_idx = hash(instruction.lens_label)
        if instruction.operation is Operation.EQUALS:
            self.add_to_box(box_idx=box_idx, lens_label=instruction.lens_label, focal_length=instruction.focal_length)
        elif instruction.operation is Operation.DASH:
            self.remove_from_box(box_idx=box_idx, lens_label=instruction.lens_label)

    def process_instruction_list(self, instruction_list: list[Instruction], verbose: bool = False) -> None:
        for instruction in instruction_list:
            if verbose:
                print(instruction)
            self.process_instruction(instruction=instruction)
            if verbose:
                self.show()
                print()

    def show(self) -> None:
        for (box_idx, box_dict) in self.items():
            if len(box_dict) > 0:
                s = f"Box {box_idx}:"
                for (lens_label, focal_length) in box_dict.items():
                    s += f' [{lens_label} {focal_length}]'
                print(s)

    def calculate_focusing_power(self) -> int:
        return sum(focusing_power(box_idx=box_idx, slot_idx=slot_idx, focal_length=focal_length)
                   for (box_idx, box_dict) in self.items()
                   for slot_idx, (lens_label, focal_length) in enumerate(box_dict.items()))

In [ ]:
b = Boxes()
instruction_list = [parse_instruction(_) for _ in parse_data(test_data)]

# b.process_instruction_list(instruction_list=instruction_list, verbose=True)
b.process_instruction_list(instruction_list=instruction_list, verbose=False)

assert b.calculate_focusing_power() == 145

In [ ]:
def q_2(data):
    b = Boxes()
    instruction_list = [parse_instruction(_) for _ in parse_data(data)]
    b.process_instruction_list(instruction_list=instruction_list, verbose=False)
    return b.calculate_focusing_power()

#### tests

In [ ]:
def test_q_2():
    LOGGER.setLevel(logging.DEBUG)
    assert q_2(test_data) == 145
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_2()

#### answer

In [ ]:
q_2(load_data())

fin